In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import numpy as np
import magic as magic
import scprep
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


sys.path.append(os.getcwd())

In [15]:
x = np.transpose(np.load('../SERGIO/imputation_data/DS6_clean_counts.npy'))
y = np.transpose(np.load('../SERGIO/imputation_data/DS6_45.npy'))
print(x.shape, y.shape)

(2700, 1200) (2700, 1200)


In [16]:
#scprep.plot.plot_library_size(x, cutoff=5000, log=True)

#### Data filtering and normalization per MAGIC repository

In [17]:
#x_hat = scprep.filter.filter_rare_genes(x, cutoff=1000, min_cells=10)
x_norm = scprep.normalize.library_size_normalize(x)
x_norm = scprep.transform.sqrt(x_norm)
x_norm.shape

(2700, 1200)

#### Run MAGIC

In [18]:
magic_op = magic.MAGIC(
    knn=5,
    knn_max=None,
    decay=1,
    t=3,
    n_pca=100,
    solver="exact",
    knn_dist="euclidean",
    n_jobs=1,
    random_state=None,
    verbose=1,
)

In [23]:
y_hat = magic_op.fit_transform(y, genes='all_genes')
y_hat.shape

Calculating MAGIC...
  Running MAGIC on 2700 cells and 1200 genes.


  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 8.28 seconds.
    Calculating KNN search...
    Calculated KNN search in 0.78 seconds.
    Calculating affinities...
    Calculated affinities in 0.53 seconds.
  Calculated graph and diffusion operator in 9.62 seconds.
  Calculating imputation...
  Calculated imputation in 0.12 seconds.
Calculated MAGIC in 9.76 seconds.


(2700, 1200)

In [24]:
np.save('yhat_MAGIC', y_hat)